In [ ]:
import os 
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings
import cloud_dic
import utils
import test_module
import brewtools

In [ ]:
#instrument

fwhm=-1
wavelength_range=[1,2.8]
ndata=3
Rfile='nonuniformR.dat'



#retrieval_params

##gas
chemeq=0
# gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gaslist = ['h2o','ch4']
gastype_list=['U','U']
ptype=77  

## clouds

do_clouds=1
npatches=1

cloudpath='../MieCode/'


cloud_name=['MgSiO3.mieff','Fe.mieff']
cloud_type=['slab','deck']

cloudpacth_index=[[1],[1]]
particle_dis=['hansan','hansan']



# cloud_name=['powerlaw', 'grey', 'ZnS_WS15.mieff']
# cloud_type=["Deck","slab","deck"]

# cloudpacth_index=[[1],[1],[1]]
# particle_dis=['hansan', 'hansan', 'hansan']


# ModelConfig:
do_fudge=1
samplemode='mcmc'

# instrument_instance = utils_dev.Instrument(fwhm,wavelength_range,ndata,Rfile)
# re_params = utils_dev.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index,particle_dis)
# model_config_instance = utils_dev.ModelConfig(samplemode,do_fudge)
# io_config_instance = utils_dev.IOConfig()


instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata,Rfile)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloud_name,cloud_type,cloudpacth_index,particle_dis)
model_config_instance = utils.ModelConfig(samplemode,do_fudge,cloudpath=cloudpath)
io_config_instance = utils.IOConfig()





model_config_instance.dist= 5.84
model_config_instance.xlist ='data/gaslistRox.dat'
model_config_instance.xpath ='/Volumes/DudleyDisk/LineLists/'
model_config_instance.do_bff=0
model_config_instance.malk=1
model_config_instance.ch4=0
model_config_instance.update_dictionary()


In [ ]:
obspec = np.asfortranarray(np.loadtxt("./examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true')) # G570D_2MassJcalib.dat
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [ ]:
# args_instance.gasnames

In [ ]:
args_instance.cloudflag

In [ ]:
np.shape(args_instance.cloudata),np.shape(args_instance.miewave),np.shape(args_instance.mierad)

In [ ]:
args_instance.cloudata[0,0,0],np.shape(args_instance.cloudata[0,0,0])

In [ ]:
args_instance=settings.runargs
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [ ]:
p0

In [ ]:
print(args_instance.cloudflag.flags['F_CONTIGUOUS'])
print(args_instance.cloudata.flags['F_CONTIGUOUS']) 
print(args_instance.miewave.flags['F_CONTIGUOUS']) 
print(args_instance.mierad.flags['F_CONTIGUOUS'])


In [ ]:
gnostics = 0

shiftspec, photspec, tauspec,cfunc = test_module.modelspec(p0[0,:],re_params,args_instance,gnostics)


In [ ]:
outspec,tmpclphotspec,tmpophotspec,cf = forwardmodel.marv(temp,logg,R2D2,gasnames,gasmass,logVMR,pcover,
do_clouds,cloudflag,cloudata,miewave,mierad,cloudrad,cloudsig,cloudprof,inlinetemps,press,inwavenum,linelist,\
cia,ciatemps,use_disort,clphot,ophot,make_cf,do_bff,bff)

    

In [ ]:
forwardmodel.marv(temp,logg,R2D2,gasnum,logVMR,pcover,do_clouds,cloudnum,cloudrad,cloudsig,cloudprof,
                  inlinetemps,press,inwavenum,linelist,cia,ciatemps,use_disort,clphot,ophot,make_cf,do_bff,bff)

In [ ]:
subroutine marv(temp,logg,R2D2,ingasname,molmass,logVMR,pcover,&
     do_clouds,incloudname,clouddata,miewave,mierad, &
     cloudrad,cloudsig,cloudprof,&
     inlinetemps,inpress,inwavenum,inlinelist,cia,ciatemps,&
     use_disort,make_cl_pspec,make_oth_pspec,make_cf,do_bff,bff,outspec,&
     cl_phot_press,oth_phot_press,cfunc)

In [ ]:
miewave,mierad,qscat,qext,cos_qscat = brewtools.pickle_load('../MieCode/MgSiO3.mieff.pic')

In [ ]:
print(mierad.shape)